In [1]:
import sys
import os
import keras_tuner
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Layer, Dropout
from tensorflow.keras import layers
from tensorflow.keras.layers import PReLU


# Get the absolute path of the current script's directory
current_dir = os.path.dirname(os.path.abspath("transformer0.ipynb"))

# Get the absolute path of the parent directory (project_folder)
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the Python path
sys.path.append(parent_dir)

from FNN1_1 import baseline_deviation, baeline_out_deviation, baseline_long_deviation, baseline_relError, absSum
baseline_out_deviation = baeline_out_deviation

from GetXY import x_train, y_train, x_val, y_val, x_test, y_test, out_x_test, out_y_test, long_x_test, long_y_test, outsideExpr, absSum

C:\Users\A_118784\Desktop\matura_github\myenv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\A_118784\Desktop\matura_github\myenv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\A_118784\Desktop\matura_github\myenv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. P

Epoch 1/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 21.9847 - val_loss: 17.2592
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 16.2054 - val_loss: 15.5965
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 14.7050 - val_loss: 13.9104
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.7592 - val_loss: 11.6461
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10.2100 - val_loss: 8.8115
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.2941 - val_loss: 5.8283
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.5001 - val_loss: 3.2360
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.4339 - val_loss: 1.6932
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.3734 - val_loss: 1.0728
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9625 - val_loss: 0.8508
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7837 - val_loss: 0.7188
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - 

C:\Users\A_118784\Desktop\matura_github\matura\matura\FNN1_1.py:246: RuntimeWarning: divide by zero encountered in divide
  relativeError = np.where(np.array(y_test) != 0, deviation.flatten() / np.abs(np.array(y_test)), deviation.flatten())


2 - -4 + -4
2543
2.0

Expressions not in x:
2 + 0 - -2
True
1457
4.0
15
-4.0
[-5.   1.   1.   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5]


In [2]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.0001,
    restore_best_weights=True,
    monitor='val_loss',
    mode = "min"
)

In [3]:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)

In [4]:
input_shape = x_train[0].shape
from tensorflow.keras import backend as K

def build_model(hp, input_shape):
    K.clear_session()
    num_neurons = hp.Int("num_neurons", 1, 512)
    num_layers = hp.Int("num_layers", 1, 16)
    dropoutTF = hp.Boolean("dropoutT/F")
    lil_model = keras.Sequential()
    lil_model.add(keras.Input(shape=input_shape))
    lil_model.add(layers.Flatten())
    for i in range(num_layers):
        lil_model.add(layers.Dense(num_neurons)),
        lil_model.add(PReLU())
        if dropoutTF == True:
            lil_model.add(layers.Dropout(0.1))
    lil_model.add(layers.Dense(1, activation='linear'))
    lil_model.compile(optimizer="adam", loss="mse")
    return lil_model
build_model(keras_tuner.HyperParameters(), input_shape)

<Sequential name=sequential, built=True>

In [5]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=lambda hp: build_model(hp, input_shape),    objective="val_loss",
    max_trials=100,
    executions_per_trial=1,
    overwrite=False,
    directory="FNNTuner",
    project_name="tuner",
)

Reloading Tuner from FNNTuner\tuner\tuner0.json


In [6]:
num_epochs = 100
tuner.search(train_dataset, epochs = num_epochs, validation_data = (val_dataset), verbose = 1, callbacks = [])

In [7]:
best_hps = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hps, input_shape)
best_model.summary()
tuner.results_summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 15)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 345)                 │           5,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ p_re_lu (PReLU)                      │ (None, 345)                 │             345 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             346 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,211 (24.26 KB)

 Trainable params: 6,211 (24.26 KB)

 Non-trainable params: 0 (0.00 B)

Results summary
Results in FNNTuner\tuner
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 025 summary
Hyperparameters:
num_neurons: 345
num_layers: 1
dropoutT/F: False
Score: 0.0012616724707186222

Trial 000 summary
Hyperparameters:
num_neurons: 348
num_layers: 1
dropoutT/F: False
Score: 0.001294966321438551

Trial 056 summary
Hyperparameters:
num_neurons: 439
num_layers: 4
dropoutT/F: False
Score: 0.0025118887424468994

Trial 020 summary
Hyperparameters:
num_neurons: 16
num_layers: 1
dropoutT/F: False
Score: 0.002628110349178314

Trial 026 summary
Hyperparameters:
num_neurons: 289
num_layers: 2
dropoutT/F: False
Score: 0.002691656118258834

Trial 042 summary
Hyperparameters:
num_neurons: 435
num_layers: 4
dropoutT/F: False
Score: 0.0027040746062994003

Trial 094 summary
Hyperparameters:
num_neurons: 440
num_layers: 4
dropoutT/F: False
Score: 0.002829659730195999

Trial 064 summary
Hyperparameters:
num_neurons: 440
num_layers: 4
dropoutT/F: False
Score: 0.0029

In [8]:
x_test_dataset = tf.data.Dataset.from_tensor_slices(x_test).batch(batch_size)
out_x_test_dataset = tf.data.Dataset.from_tensor_slices(out_x_test).batch(batch_size)
long_x_test_dataset = tf.data.Dataset.from_tensor_slices(long_x_test).batch(batch_size)

In [9]:
best_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 18.1515 - val_loss: 14.9514
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 14.6610 - val_loss: 12.6578
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.0870 - val_loss: 9.7645
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.5846 - val_loss: 6.4104
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.9475 - val_loss: 3.2383
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.3040 - val_loss: 1.3889
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0911 - val_loss: 0.7940
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6729 - val_loss: 0.5685
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4775 - val_loss: 0.4032
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3577 - val_loss: 0.3168
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2663 - val_loss: 0.2372
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss

In [10]:
predsInRange = best_model.predict(x_test_dataset)
predsOutRange = best_model.predict(out_x_test_dataset)
predsLongRange = best_model.predict(long_x_test_dataset)

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [11]:
import numpy as np

reldiffInRange = []
diffInRange = []
safe_y_test = np.where(np.isclose(y_test,0.0), 1.0, y_test)

for i in range(len(y_test)):
    diffInRange.append(abs(y_test[i] - predsInRange[i]))
    reldiffInRange.append(abs(y_test[i] - predsInRange[i])/abs(safe_y_test[i]))
print(len(diffInRange))
print("MAE in Range: ", np.mean(diffInRange))
print("MRE in Range: ", np.mean(reldiffInRange))

diffLongRange = []
for i in range(200, 300):
    diffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsLongRange[i])))
    
NEEDdiffLongRange = []
for i in range(len(long_y_test)):
    NEEDdiffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsLongRange[i])))
print("MAE longer Expressions: ", np.mean(NEEDdiffLongRange))

diffOutRange = []
for i in range(len(out_y_test)):
    diffOutRange.append(abs(out_y_test[i] - predsOutRange[i]))
safe_out_y_test = np.where(out_y_test == 0, 1, out_y_test)
diff_out_relError = []
for i in range(len(out_y_test)):
    diff_out_relError.append(abs(diffOutRange[i] / safe_out_y_test[i]))
print("MAE out Range: ", np.mean(diffOutRange))
print("MRE out Range: ", np.mean(diff_out_relError))

1457
MAE in Range:  0.024680045
MRE in Range:  0.009897356833310939
MAE longer Expressions:  5.4590703339755
MAE out Range:  2.2411294
MRE out Range:  0.26136130026143933


In [12]:
placeholder = absSum(outsideExpr)
diffOutRange = []
indices_with_placeholder_22 = [i for i, val in enumerate(placeholder) if val == 22] 

for i in indices_with_placeholder_22:
    diffOutRange.append(np.abs(out_y_test[i]-predsOutRange[i]))


In [13]:
meanDiff_InRange = np.mean(diffInRange)
meanDiff_OutRange = np.mean(diffOutRange)
meanDiff_LongRange = np.mean(diffLongRange)
meanDiff_OutRelRange = np.mean(diff_out_relError)



In [14]:
benchmark = 0
benchmark += baseline_deviation / (meanDiff_InRange**2) / 4
print(baseline_deviation / (meanDiff_InRange**2) / 4)

benchmark += baseline_out_deviation / (meanDiff_OutRange**2) / 4
print(baseline_out_deviation / (meanDiff_OutRange**2) / 4)

benchmark += baseline_long_deviation / (meanDiff_LongRange**2) / 4
print(baseline_long_deviation / (meanDiff_LongRange**2) / 4)

benchmark += baseline_relError / (meanDiff_OutRelRange**2) / 4
print(baseline_relError / (meanDiff_OutRelRange**2) / 4)

print(f"Benchmark: {benchmark}")

11.530600492619657
0.4261588271210863
0.20455705161488497
0.21937706643338284
Benchmark: 12.380693437789011


In [15]:
# New cell for RNN2 notebook - Statistical analysis with multiple runs
#this is also generated based on FNN6.py (DeepseekR1 this time.) (ofc, slightly tweaked as always)
# Initialize lists to store metrics across multiple runs
benchmarks_rnn = []
MAEinRange_rnn = []
MREinRange_rnn = []
MAEoutRange_rnn = []
MREoutRange_rnn = []
MAElongRange_rnn = []

# Run multiple training iterations
for progress in range(5):
    early_stopping = keras.callbacks.EarlyStopping(
        patience=10,
        min_delta=0.0001,
        restore_best_weights=True,
        monitor='val_loss',
        mode = "min"
    )
    print(f"Progress: {progress + 1}/5")
    
    # Build and train model
    current_model = build_model(best_hps, input_shape)
    
    current_model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=200,
        callbacks=[early_stopping],
        verbose=1
    )
    
    # Make predictions
    predsInRange = current_model.predict(x_test_dataset, verbose=0)
    predsOutRange = current_model.predict(out_x_test_dataset, verbose=0)
    predsLongRange = current_model.predict(long_x_test_dataset, verbose=0)
    
    # Calculate metrics
    reldiffInRange = []
    diffInRange = []
    safe_y_test = np.where(np.isclose(y_test, 0.0), 1.0, y_test)
    
    for i in range(len(y_test)):
        diffInRange.append(abs(y_test[i] - predsInRange[i]))
        reldiffInRange.append(abs(y_test[i] - predsInRange[i])/abs(safe_y_test[i]))
    
    MAEinRange_rnn.append(np.mean(diffInRange))
    MREinRange_rnn.append(np.mean(reldiffInRange))

    # Long range metrics
    diffLongRange = []
    for i in range(200, 300):
        diffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsLongRange[i])))
    
    NEEDdiffLongRange = []
    for i in range(len(long_y_test)):
        NEEDdiffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsLongRange[i])))
    MAElongRange_rnn.append(np.mean(NEEDdiffLongRange))
    
    # Out of range metrics
    diffOutRange = []
    for i in range(len(out_y_test)):
        diffOutRange.append(abs(out_y_test[i] - predsOutRange[i]))
    safe_out_y_test = np.where(out_y_test == 0, 1, out_y_test)
    diff_out_relError = []
    for i in range(len(out_y_test)):
        diff_out_relError.append(abs(diffOutRange[i] / safe_out_y_test[i]))
    
    MAEoutRange_rnn.append(np.mean(diffOutRange))
    MREoutRange_rnn.append(np.mean(diff_out_relError))
    
    # Calculate benchmark
    placeholder = absSum(outsideExpr)
    diffOutRange_22 = []
    indices_with_placeholder_22 = [i for i, val in enumerate(placeholder) if val == 22] 
    
    for i in indices_with_placeholder_22:
        diffOutRange_22.append(np.abs(out_y_test[i]-predsOutRange[i]))
    
    meanDiff_InRange = np.mean(diffInRange)
    meanDiff_OutRange = np.mean(diffOutRange_22)
    meanDiff_LongRange = np.mean(diffLongRange)
    meanDiff_OutRelRange = np.mean(diff_out_relError)
    
    benchmark = 0
    benchmark += baseline_deviation / (meanDiff_InRange**2) / 4
    benchmark += baseline_out_deviation / (meanDiff_OutRange**2) / 4
    benchmark += baseline_long_deviation / (meanDiff_LongRange**2) / 4
    benchmark += baseline_relError / (meanDiff_OutRelRange**2) / 4
    
    benchmarks_rnn.append(benchmark)

# Statistical analysis
from scipy.stats import ttest_1samp


stats6, p_value6 = ttest_1samp(benchmarks_rnn, popmean=1)

print(f"FNN Model - Statistical Analysis Results:")

print(f"Benchmark P-value: {p_value6:.6f}")

print(f"\nFNN Model - Average Metrics:")
print(f"Average MAE in Range: {np.mean(MAEinRange_rnn):.6f}")
print(f"Average MRE in Range: {np.mean(MREinRange_rnn):.6f}")
print(f"Average MAE out Range: {np.mean(MAEoutRange_rnn):.6f}")
print(f"Average MRE out Range: {np.mean(MREoutRange_rnn):.6f}")
print(f"Average MAE long Range: {np.mean(MAElongRange_rnn):.6f}")
print(f"Average benchmark: {np.mean(benchmarks_rnn):.6f}")

print(f"\nFNN Model - All Runs:")
print(f"MAE in Range: {[f'{x:.6f}' for x in MAEinRange_rnn]}")
print(f"MRE in Range: {[f'{x:.6f}' for x in MREinRange_rnn]}")
print(f"MAE out Range: {[f'{x:.6f}' for x in MAEoutRange_rnn]}")
print(f"MRE out Range: {[f'{x:.6f}' for x in MREoutRange_rnn]}")
print(f"MAE long Range: {[f'{x:.6f}' for x in MAElongRange_rnn]}")
print(f"Benchmark: {[f'{x:.6f}' for x in benchmarks_rnn]}")

Progress: 1/5
Epoch 1/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 18.3982 - val_loss: 15.0893
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 14.9472 - val_loss: 12.8441
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.2860 - val_loss: 9.9796
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.8040 - val_loss: 6.7087
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.1657 - val_loss: 3.3204
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.3625 - val_loss: 1.3629
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1193 - val_loss: 0.7659
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6897 - val_loss: 0.5771
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4973 - val_loss: 0.4101
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3670 - val_loss: 0.3330
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2792 - val_loss: 0.2656
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3